<a href="https://colab.research.google.com/github/gtc0823/RAG/blob/main/RAG01_%E6%89%93%E9%80%A0%E5%90%91%E9%87%8F%E8%B3%87%E6%96%99%E5%BA%AB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 1. 建立資料夾

In [ ]:
import os
upload_dir = "uploaded_docs"
os.makedirs(upload_dir, exist_ok=True)
print(f"請將你的 .txt, .pdf, .docx 檔案放到這個資料夾中： {upload_dir}")

<font color="red">請手動上傳自己的檔案再繼續。</font>

### 2. 更新必要套件並引入

In [ ]:
!pip install -U langchain langchain-community pypdf python-docx sentence-transformers faiss-cpu

In [ ]:
from langchain_community.document_loaders import TextLoader, PyPDFLoader, UnstructuredWordDocumentLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

### 3. 依 e5 建議加入

自訂支援 E5 的 embedding 模型（加上 "passage:" / "query:" 前綴）

可以直接使用 E5 模型做查詢與文件的語意比對，而不需要在每次呼叫時都手動加上 "query: " 或 "passage: "

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings

class CustomE5Embedding(HuggingFaceEmbeddings):
    def embed_documents(self, texts):
        texts = [f"passage: {t}" for t in texts]
        return super().embed_documents(texts)

    def embed_query(self, text):
        return super().embed_query(f"query: {text}")

### 4. 載入文件

In [ ]:
import json
from langchain.schema import Document

In [ ]:
folder_path = upload_dir
documents = []
for file in os.listdir(folder_path):
    path = os.path.join(folder_path, file)
    if file.endswith(".txt"):
        loader = TextLoader(path)
    elif file.endswith(".pdf"):
        loader = PyPDFLoader(path)
    elif file.endswith(".docx"):
        loader = UnstructuredWordDocumentLoader(path)
    else:
        continue
    documents.extend(loader.load())

### 5. 建立向量資料庫

把一整份的 documents 切成小段


*   chunk_size=500：每段最多 500 個字元。
*   chunk_overlap=50：每段之間重疊 50 個字元（避免切斷語意）







In [ ]:
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
split_docs = splitter.split_documents(documents)

嵌入模型：CustomE5Embedding

建立向量資料庫（FAISS）

將分段後的 split_docs 經過嵌入模型轉成向量，再餵進 FAISS 建立索引，之後就能用來做快速的語意搜尋

FAISS 是 Facebook 提供的高效「相似度搜尋引擎」，能在大規模向量中快速找出最相近的項目

In [ ]:
embedding_model = CustomE5Embedding(model_name="intfloat/multilingual-e5-small")
vectorstore = FAISS.from_documents(split_docs, embedding_model)

### 6. 儲存向量資料庫

In [ ]:
vectorstore.save_local("faiss_db")

In [ ]:
!zip -r faiss_db.zip faiss_db

In [ ]:
print("✅ 壓縮好的向量資料庫已儲存為 'faiss_db.zip'，請下載此檔案備份。")